# Titanic - Part 1: Data Cleaning

## 1. Importing Dataset and Necessary Packages

In [1]:
import numpy as np                                  # linear algebra
import os                                           # package used to set work directory
import pandas as pd                                 # package used to create dataframes
import re                                           # regular expressions

from operator import itemgetter                     # package used for sorted, max, min 

In [2]:
data = pd.read_csv('test.csv')
df_test = data.copy()

data = pd.read_csv('train.csv')
df_train = data.copy()

In [3]:
df_test['Nationality'] = pd.read_csv('nationality/testNationalitySubset.csv')['Nationality']
df_train['Nationality'] = pd.read_csv('nationality/trainNationalitySubset.csv')['Nationality']

In [4]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
 11  Nationality  418 non-null    object 
dtypes: float64(2), int64(4), object(6)
memory usage: 39.3+ KB


In [5]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
 12  Nationality  891 non-null    object 
dtypes: float64(2), int64(5), object(6)
memory usage: 90.6+ KB


Both the test and training set have missing values. The data types look fine.

In [6]:
print(any(df_test.duplicated()), any(df_train.duplicated()))

False False


There are no duplicates.

## 2. Data Cleaning

### 2.1 Missing Values

In the previous section, we already checked for duplicates and missing values. There were no duplicates, but there were missing values. In this subsection, we will take a look at the missing values in both the test set and the training set.

#### 2.1.1 Test Set

We're not going to go too deep into the test set as this set will only be used to test how well the model is performing. We just need to make sure that the data set won't cause issues when used in later applications.

In [7]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
 11  Nationality  418 non-null    object 
dtypes: float64(2), int64(4), object(6)
memory usage: 39.3+ KB


There are some columns that contain NULL elements. This has to be addressed.

In [8]:
df_test[df_test['Fare'].isna()]

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Nationality
152,1044,3,"Storey, Mr. Thomas",male,60.5,0,0,3701,NaN,NaN,S,CelticEnglish


There is one passenger for whom the fare is not known. We can try to make an educated guess for how much this passenger's fare was or we could simply drop this passenger. We're dealing with a 60 year old man named Thomas Storey. Let's see if there is a Mrs Storey or any other relatives with the same last name.

In [9]:
df_test.loc[[i for i in range(len(df_test.Name)) if 'Storey' in df_test.loc[i, 'Name']], 'Name']

152    Storey, Mr. Thomas
Name: Name, dtype: object

Unfortunately, there's only one passenger with this name. We can look at what others paid embarking from Southampton with a lower class ticket.

In [10]:
df_test.loc[(df_test['Pclass'] == 3) & (df_test['Embarked'] == 'S') & (df_test['Sex'] == 'male')].head(10)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Nationality
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,"European,German"
5,897,3,"Svensson, Mr. Johan Cervin",male,14.0,0,0,7538,9.2250,NaN,S,"Nordic,Scandinavian,Sweden"
9,901,3,"Davies, Mr. John Samuel",male,21.0,2,0,A/4 48871,24.1500,NaN,S,CelticEnglish
10,902,3,"Ilieff, Mr. Ylio",male,NaN,0,0,349220,7.8958,NaN,S,"Hispanic,Spanish"
21,913,3,"Olsen, Master. Artur Karl",male,9.0,0,1,C 17368,3.1708,NaN,S,"Nordic,Scandinavian,Norway"
25,917,3,"Robins, Mr. Alexander A",male,50.0,1,0,A/5. 3337,14.5000,NaN,S,CelticEnglish
38,930,3,"Sap, Mr. Julius",male,25.0,0,0,345768,9.5000,NaN,S,"Hispanic,Philippines"
39,931,3,"Hee, Mr. Ling",male,NaN,0,0,1601,56.4958,NaN,S,"EastAsian,Chinese"
42,934,3,"Goldsmith, Mr. Nathan",male,41.0,0,0,SOTON/O.Q. 3101263,7.8500,NaN,S,CelticEnglish
45,937,3,"Peltomaki, Mr. Nikolai Johannes",male,25.0,0,0,STON/O 2. 3101291,7.9250,NaN,S,"European,German"


The fare seems to vary a lot even within this subgroup. Maybe the ticket number can tell us something about the fare. We will look into all passenger that had a four digit ticket number.

In [11]:
df_test_4digits = df_test.loc[[i for i in range(len(df_test['Ticket'])) if len(df_test.loc[i, 'Ticket']) == 4]]
df_test_4digits

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Nationality
5,897,3,"Svensson, Mr. Johan Cervin",male,14.0,0,0,7538,9.2250,NaN,S,"Nordic,Scandinavian,Sweden"
8,900,3,"Abrahim, Mrs. Joseph (Sophie Halaut Easu)",female,18.0,0,0,2657,7.2292,NaN,C,CelticEnglish
17,909,3,"Assaf, Mr. Gerios",male,21.0,0,0,2692,7.2250,NaN,C,"Muslim,Nubian"
19,911,3,"Assaf Khalil, Mrs. Mariana (Miriam"")""",female,45.0,0,0,2696,7.2250,NaN,C,"Muslim,Nubian"
27,919,3,"Daher, Mr. Shedid",male,22.5,0,0,2698,7.2250,NaN,C,"Muslim,Nubian"
29,921,3,"Samaan, Mr. Elias",male,NaN,2,0,2662,21.6792,NaN,C,"Muslim,Nubian"
35,927,3,"Katavelas, Mr. Vassilios (Catavelas Vassilios"")""",male,18.5,0,0,2682,7.2292,NaN,C,Greek
39,931,3,"Hee, Mr. Ling",male,NaN,0,0,1601,56.4958,NaN,S,"EastAsian,Chinese"
68,960,1,"Tucker, Mr. Gilbert Milligan Jr",male,31.0,0,0,2543,28.5375,C53,C,CelticEnglish
80,972,3,"Boulos, Master. Akar",male,6.0,1,1,2678,15.2458,NaN,C,"Muslim,Nubian"


This doesn't really tell us a lot about the fare either.

In [12]:
df_test_4digits.loc[(df_test['Pclass'] == 3) & (df_test['Embarked'] == 'S') & (df_test['Sex'] == 'male')]

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Nationality
5,897,3,"Svensson, Mr. Johan Cervin",male,14.0,0,0,7538,9.2250,NaN,S,"Nordic,Scandinavian,Sweden"
39,931,3,"Hee, Mr. Ling",male,NaN,0,0,1601,56.4958,NaN,S,"EastAsian,Chinese"
108,1000,3,"Willer, Mr. Aaron (Abi Weller"")""",male,NaN,0,0,3410,8.7125,NaN,S,"European,German"
152,1044,3,"Storey, Mr. Thomas",male,60.5,0,0,3701,NaN,NaN,S,CelticEnglish
243,1135,3,"Hyman, Mr. Abraham",male,NaN,0,0,3470,7.8875,NaN,S,CelticEnglish
253,1145,3,"Salander, Mr. Karl Johan",male,24.0,0,0,7266,9.3250,NaN,S,"Nordic,Scandinavian,Sweden"
357,1249,3,"Lockyer, Mr. Edward",male,NaN,0,0,1222,7.8792,NaN,S,CelticEnglish


Other male third class passengers that embarked from Southampton and had a four digit ticket number had a relatively low fare between 7.80 and 9.40 pounds. There's only one exception that paid 56.50 pounds. For now, we replace this null element by the average of these other similar passengers, except the one outlier.

In [13]:
mean_fare_storey = np.mean(df_test_4digits.loc[(df_test['Pclass'] == 3) & (df_test['Embarked'] == 'S') & 
                           (df_test['Sex'] == 'male') & (df_test['Fare'] < 50) & (~df_test['Fare'].isna()), 'Fare'])
mean_fare_storey

np.float64(8.605839999999999)

In [14]:
df_test.loc[152, 'Fare'] = mean_fare_storey
df_test.loc[152]

PassengerId                  1044
Pclass                          3
Name           Storey, Mr. Thomas
Sex                          male
Age                          60.5
SibSp                           0
Parch                           0
Ticket                       3701
Fare                      8.60584
Cabin                         NaN
Embarked                        S
Nationality         CelticEnglish
Name: 152, dtype: object

The column `Cabin` has only 91 non-null entries. It's not worth the time and effort to do any manipulation on this one to replace any missing values. Since we're dealing with string values, we'll replace the NULL elements by 'unknown'.

In [15]:
df_test.loc[df_test['Cabin'].isna(), 'Cabin'] = 'unknown'

There's one column left that has 86 NULL entries: `Age`. There is no direct solution to these missing values and we would rather not get rid of so many passengers (21 percent of the test set!). Later on we will fill these missing entries in the modeling process, but for now we will leave it as it is. We will add a column that indicates whether an individual has a missing entry for age. This will come in handy later.

In [16]:
df_test['Missing_Age'] = 0
df_test.loc[df_test['Age'].isna(), 'Missing_Age'] = 1

In [17]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         418 non-null    float64
 9   Cabin        418 non-null    object 
 10  Embarked     418 non-null    object 
 11  Nationality  418 non-null    object 
 12  Missing_Age  418 non-null    int64  
dtypes: float64(2), int64(5), object(6)
memory usage: 42.6+ KB


#### 2.1.2 Training set

The training set will be used to train our classification model.

In [18]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
 12  Nationality  891 non-null    object 
dtypes: float64(2), int64(5), object(6)
memory usage: 90.6+ KB


Again, we see missing values for the columns `Age` and `Cabin`, but now there are also a few values missing in the column `Embarked`. As opposed to the test set, there are no missing values for `Fare`. There are two missing values for `Embarked`. We'll look into these.

In [19]:
df_train.loc[df_train['Embarked'].isna()]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Nationality
61,62,1,1,"Icard, Miss. Amelie",female,38.0,0,0,113572,80.0,B28,NaN,"European,French"
829,830,1,1,"Stone, Mrs. George Nelson (Martha Evelyn)",female,62.0,0,0,113572,80.0,B28,NaN,CelticEnglish


We are dealing with two women who happen to have the same ticket number and the same cabin number. One of them is unmarried and the other is married. We can try find relatives of the married woman. This could have been a woman traveling with her maid. They had no other relatives on the ship.

We can try the cabin number.

In [20]:
[i for i in range(len(df_train)) if df_train.loc[i, 'Cabin'] == 'B28']

[61, 829]

They were really alone in that cabin. Maybe the fare can help us.

In [21]:
df_train.loc[[i for i in range(len(df_train)) if (df_train.loc[i, 'Fare'] > 75) & (df_train.loc[i, 'Fare'] < 90)]]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Nationality
34,35,0,1,"Meyer, Mr. Edgar Joseph",male,28.0,1,0,PC 17604,82.1708,NaN,C,"European,German"
52,53,1,1,"Harper, Mrs. Henry Sleeper (Myna Haxtun)",female,49.0,1,0,PC 17572,76.7292,D33,C,CelticEnglish
61,62,1,1,"Icard, Miss. Amelie",female,38.0,0,0,113572,80.0000,B28,NaN,"European,French"
62,63,0,1,"Harris, Mr. Henry Birkhardt",male,45.0,1,0,36973,83.4750,C83,S,CelticEnglish
102,103,0,1,"White, Mr. Richard Frasar",male,21.0,0,1,35281,77.2875,D26,S,CelticEnglish
124,125,0,1,"White, Mr. Percival Wayland",male,54.0,0,1,35281,77.2875,D26,S,CelticEnglish
139,140,0,1,"Giglio, Mr. Victor",male,24.0,0,0,PC 17593,79.2000,B86,C,"Hispanic,Spanish"
218,219,1,1,"Bazzani, Miss. Albina",female,32.0,0,0,11813,76.2917,D15,C,"European,Italian,Italy"
230,231,1,1,"Harris, Mrs. Henry Birkhardt (Irene Wallach)",female,35.0,1,0,36973,83.4750,C83,S,CelticEnglish
256,257,1,1,"Thorne, Mrs. Gertrude Maybelle",female,NaN,0,0,PC 17585,79.2000,NaN,C,CelticEnglish


We include the cabin number to the analysis as well.

In [22]:
df_train.loc[[i for i in range(len(df_train)) if (df_train.loc[i, 'Fare'] > 75) & (df_train.loc[i, 'Fare'] < 90) & 
              ('B' in str(df_train.loc[i, 'Cabin']))]]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Nationality
61,62,1,1,"Icard, Miss. Amelie",female,38.0,0,0,113572,80.0,B28,NaN,"European,French"
139,140,0,1,"Giglio, Mr. Victor",male,24.0,0,0,PC 17593,79.2,B86,C,"Hispanic,Spanish"
257,258,1,1,"Cherry, Miss. Gladys",female,30.0,0,0,110152,86.5,B77,S,CelticEnglish
504,505,1,1,"Maioni, Miss. Roberta",female,16.0,0,0,110152,86.5,B79,S,"Hispanic,Portuguese"
587,588,1,1,"Frolicher-Stehli, Mr. Maxmillian",male,60.0,1,1,13567,79.2,B41,C,"African,EastAfrican"
759,760,1,1,"Rothes, the Countess. of (Lucy Noel Martha Dye...",female,33.0,0,0,110152,86.5,B77,S,"Hispanic,Spanish"
789,790,0,1,"Guggenheim, Mr. Benjamin",male,46.0,0,0,PC 17593,79.2,B82 B84,C,"European,German"
829,830,1,1,"Stone, Mrs. George Nelson (Martha Evelyn)",female,62.0,0,0,113572,80.0,B28,NaN,CelticEnglish


We're going to make an educated guess and assume that these two women embarked from Southampton. Looking at the ticket number, there are only a few that have the same amount of digits and start with 11 and whose cabin number starts with a 'B'.

In [23]:
df_train.loc[[61, 829], 'Embarked'] = 'S'
df_train.loc[[61, 829]]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Nationality
61,62,1,1,"Icard, Miss. Amelie",female,38.0,0,0,113572,80.0,B28,S,"European,French"
829,830,1,1,"Stone, Mrs. George Nelson (Martha Evelyn)",female,62.0,0,0,113572,80.0,B28,S,CelticEnglish


We replace the NULL elements in `Cabin` by 'unknown'.

In [24]:
df_train.loc[df_train['Cabin'].isna(), 'Cabin'] = 'unknown'

We create a column to indicate the individuals for whom the age is not known.

In [25]:
df_train['Missing_Age'] = 0
df_train.loc[df_train['Age'].isna(), 'Missing_Age'] = 1

In [26]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        891 non-null    object 
 11  Embarked     891 non-null    object 
 12  Nationality  891 non-null    object 
 13  Missing_Age  891 non-null    int64  
dtypes: float64(2), int64(6), object(6)
memory usage: 97.6+ KB


## 2.2 Pclass

`Pclass` is the ticket class and can be seen as a proxy for socio-economic status (SES). The source of this data set says that there should be three classes. We will check if this is true. Otherwise, we would have to fix mistakes in the entries.

In [27]:
print('Unique classes in the test set:', pd.unique(df_test['Pclass']))
print('Unique classes in the train set:', pd.unique(df_train['Pclass']))

Unique classes in the test set: [3 2 1]
Unique classes in the train set: [3 1 2]


All passengers are assigned to one of the three classes.

## 2.3 Sex

There should be only male and female assigned to the passengers.

In [28]:
print('Unique sexes in the test set:', pd.unique(df_test['Sex']))
print('Unique sexes in the train set:', pd.unique(df_train['Sex']))

Unique sexes in the test set: ['male' 'female']
Unique sexes in the train set: ['male' 'female']


## 2.4 Age

In [29]:
df_test.loc[df_test['Age'] > -1, 'Age'].describe()

count    332.000000
mean      30.272590
std       14.181209
min        0.170000
25%       21.000000
50%       27.000000
75%       39.000000
max       76.000000
Name: Age, dtype: float64

The lowest age in the test set is 0.17, i.e., about two months, and the highest age is 76. There are no strange outliers.

In [30]:
df_train.loc[df_train['Age'] > -1, 'Age'].describe()

count    714.000000
mean      29.699118
std       14.526497
min        0.420000
25%       20.125000
50%       28.000000
75%       38.000000
max       80.000000
Name: Age, dtype: float64

The lowest age in the training set is 0.42, i.e., about five months, and the highest age is 80. There are no strange outliers.

## 2.5 SibSp

This feature `SibSp` represents the number of siblings or spouses were on the ship. The data type is 'int64'.

In [31]:
df_test['SibSp'].describe()

count    418.000000
mean       0.447368
std        0.896760
min        0.000000
25%        0.000000
50%        0.000000
75%        1.000000
max        8.000000
Name: SibSp, dtype: float64

The lowest number of siblings or spouses in the test set is 0 and the highest number is 8. There are no strange outliers.

In [32]:
df_train['SibSp'].describe()

count    891.000000
mean       0.523008
std        1.102743
min        0.000000
25%        0.000000
50%        0.000000
75%        1.000000
max        8.000000
Name: SibSp, dtype: float64

The lowest number of siblings or spouses in the training set is 0 and the highest number is 8. There are no strange outliers.

## 2.6 Parch

This feature `Parch` represents the number of parents or children were on the ship. The data type is 'int64'.

In [33]:
df_test['Parch'].describe()

count    418.000000
mean       0.392344
std        0.981429
min        0.000000
25%        0.000000
50%        0.000000
75%        0.000000
max        9.000000
Name: Parch, dtype: float64

The lowest number of parents or children in the test set is 0 and the highest number is 9. There are no strange outliers.

In [34]:
df_train['Parch'].describe()

count    891.000000
mean       0.381594
std        0.806057
min        0.000000
25%        0.000000
50%        0.000000
75%        0.000000
max        6.000000
Name: Parch, dtype: float64

The lowest number of parents or children in the training set is 0 and the highest number is 6. There are no strange outliers.

## 2.7 Fare

In [35]:
df_test['Fare'].describe()

count    418.000000
mean      35.562544
std       55.856139
min        0.000000
25%        7.895800
50%       14.454200
75%       31.471875
max      512.329200
Name: Fare, dtype: float64

In [36]:
df_train['Fare'].describe()

count    891.000000
mean      32.204208
std       49.693429
min        0.000000
25%        7.910400
50%       14.454200
75%       31.000000
max      512.329200
Name: Fare, dtype: float64

Apart from the zero fares, there don't seem to be any unrealistic fares. There obviously were different ticket classes and therefore different fares. Let's see what's going on with the zero fares.

In [37]:
df_test.loc[df_test['Fare'] < 1]

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Nationality,Missing_Age
266,1158,1,"Chisholm, Mr. Roderick Robert Crispin",male,NaN,0,0,112051,0.0,unknown,S,CelticEnglish,1
372,1264,1,"Ismay, Mr. Joseph Bruce",male,49.0,0,0,112058,0.0,B52 B54 B56,S,CelticEnglish,0


In [38]:
df_train.loc[df_train['Fare'] < 1]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Nationality,Missing_Age
179,180,0,3,"Leonard, Mr. Lionel",male,36.0,0,0,LINE,0.0,unknown,S,"European,French",0
263,264,0,1,"Harrison, Mr. William",male,40.0,0,0,112059,0.0,B94,S,CelticEnglish,0
271,272,1,3,"Tornquist, Mr. William Henry",male,25.0,0,0,LINE,0.0,unknown,S,"Hispanic,Spanish",0
277,278,0,2,"Parkes, Mr. Francis ""Frank""",male,NaN,0,0,239853,0.0,unknown,S,CelticEnglish,1
302,303,0,3,"Johnson, Mr. William Cahoone Jr",male,19.0,0,0,LINE,0.0,unknown,S,CelticEnglish,0
413,414,0,2,"Cunningham, Mr. Alfred Fleming",male,NaN,0,0,239853,0.0,unknown,S,CelticEnglish,1
466,467,0,2,"Campbell, Mr. William",male,NaN,0,0,239853,0.0,unknown,S,CelticEnglish,1
481,482,0,2,"Frost, Mr. Anthony Wood ""Archie""",male,NaN,0,0,239854,0.0,unknown,S,CelticEnglish,1
597,598,0,3,"Johnson, Mr. Alfred",male,49.0,0,0,LINE,0.0,unknown,S,CelticEnglish,0
633,634,0,1,"Parr, Mr. William Henry Marsh",male,NaN,0,0,112052,0.0,unknown,S,CelticEnglish,1


All the zero fares are from passengers that embarked from Southampton. After googling some of these names, it became clear that these were mainly crew members and invitees.

## 2.8 Embarked

`Embarked` shows from which port the passenger embarked. There are three ports from which passengers could embark: Cherbourg (C), Queenstown (Q) and Southampton (S).

In [39]:
print('Unique ports in the test set:', pd.unique(df_test['Embarked']))
print('Unique ports in the train set:', pd.unique(df_train['Embarked']))

Unique ports in the test set: ['Q' 'S' 'C']
Unique ports in the train set: ['S' 'C' 'Q']


All passengers are assigned to one of the three ports.

## 2.9 Cabin

In [40]:
pd.unique(df_test['Cabin'])

array(['unknown', 'B45', 'E31', 'B57 B59 B63 B66', 'B36', 'A21', 'C78',
       'D34', 'D19', 'A9', 'D15', 'C31', 'C23 C25 C27', 'F G63', 'B61',
       'C53', 'D43', 'C130', 'C132', 'C101', 'C55 C57', 'B71', 'C46',
       'C116', 'F', 'A29', 'G6', 'C6', 'C28', 'C51', 'E46', 'C54', 'C97',
       'D22', 'B10', 'F4', 'E45', 'E52', 'D30', 'B58 B60', 'E34',
       'C62 C64', 'A11', 'B11', 'C80', 'F33', 'C85', 'D37', 'C86', 'D21',
       'C89', 'F E46', 'A34', 'D', 'B26', 'C22 C26', 'B69', 'C32', 'B78',
       'F E57', 'F2', 'A18', 'C106', 'B51 B53 B55', 'D10 D12', 'E60',
       'E50', 'E39 E41', 'B52 B54 B56', 'C39', 'B24', 'D28', 'B41', 'C7',
       'D40', 'D38', 'C105'], dtype=object)

In [41]:
df_test['Cabin'].value_counts()

Cabin
unknown            327
B57 B59 B63 B66      3
B45                  2
C23 C25 C27          2
C78                  2
                  ... 
B41                  1
C7                   1
D40                  1
D38                  1
C105                 1
Name: count, Length: 77, dtype: int64

In [42]:
pd.unique(df_train['Cabin'])

array(['unknown', 'C85', 'C123', 'E46', 'G6', 'C103', 'D56', 'A6',
       'C23 C25 C27', 'B78', 'D33', 'B30', 'C52', 'B28', 'C83', 'F33',
       'F G73', 'E31', 'A5', 'D10 D12', 'D26', 'C110', 'B58 B60', 'E101',
       'F E69', 'D47', 'B86', 'F2', 'C2', 'E33', 'B19', 'A7', 'C49', 'F4',
       'A32', 'B4', 'B80', 'A31', 'D36', 'D15', 'C93', 'C78', 'D35',
       'C87', 'B77', 'E67', 'B94', 'C125', 'C99', 'C118', 'D7', 'A19',
       'B49', 'D', 'C22 C26', 'C106', 'C65', 'E36', 'C54',
       'B57 B59 B63 B66', 'C7', 'E34', 'C32', 'B18', 'C124', 'C91', 'E40',
       'T', 'C128', 'D37', 'B35', 'E50', 'C82', 'B96 B98', 'E10', 'E44',
       'A34', 'C104', 'C111', 'C92', 'E38', 'D21', 'E12', 'E63', 'A14',
       'B37', 'C30', 'D20', 'B79', 'E25', 'D46', 'B73', 'C95', 'B38',
       'B39', 'B22', 'C86', 'C70', 'A16', 'C101', 'C68', 'A10', 'E68',
       'B41', 'A20', 'D19', 'D50', 'D9', 'A23', 'B50', 'A26', 'D48',
       'E58', 'C126', 'B71', 'B51 B53 B55', 'D49', 'B5', 'B20', 'F G63',
       'C62

In [43]:
df_train['Cabin'].value_counts()

Cabin
unknown        687
G6               4
C23 C25 C27      4
B96 B98          4
F2               3
              ... 
E17              1
A24              1
C50              1
B42              1
C148             1
Name: count, Length: 148, dtype: int64

The letters in these cabin numbers represent the deck. There were 8 decks. Notice that some cabin numbers contain two letters. This is the case for deck F. The second letter represents the section on this deck. We can create an extra column for the deck. 

Source: https://www.encyclopedia-titanica.org/cabins.html

In [44]:
df_test['Deck'] = [df_test.loc[i, 'Cabin'][0] for i in range(len(df_test))]
df_test.loc[df_test['Deck'] == 'u', 'Deck'] = 'unknown'

In [45]:
df_train['Deck'] = [df_train.loc[i, 'Cabin'][0] for i in range(len(df_train))]
df_train.loc[df_train['Deck'] == 'u', 'Deck'] = 'unknown'

## 2.10 Name

There is a lot that can be deduced from the name of a passenger. We'll begin with the title.

In [46]:
pd.DataFrame([name.split(', ', 1)[1].split(' ', 1)[0] for name in df_train['Name']]).value_counts()

0        
Mr.          517
Miss.        182
Mrs.         125
Master.       40
Dr.            7
Rev.           6
Mlle.          2
Major.         2
Col.           2
Capt.          1
Lady.          1
Don.           1
Jonkheer.      1
Mme.           1
Ms.            1
Sir.           1
the            1
Name: count, dtype: int64

There is a title that starts with 'the'. Because of how these titles are extracted, we need to do some extra work on this title.

In [47]:
[name for name in df_train['Name'] if name.split(', ', 1)[1].split(' ', 1)[0] == 'the']

['Rothes, the Countess. of (Lucy Noel Martha Dyer-Edwards)']

There is one passenger with the title 'the Countess of'.

"Don" is a Spanish title for lords. "Jonkheer" is a Dutch title of nobility. "Mlle." is the French version of "Miss." and "Mme." of "Mrs.". The other titles are English and speak for themselves.

We can check some entries to see if there are mistakes. We can start with the sex.

In [48]:
titles = set([name.split(', ', 1)[1].split(' ', 1)[0] for name in df_train['Name']])

def unique_sexes_titles(title):
    print('Title:', title)
    print(set([df_train.loc[i, 'Sex'] for i in range(len(df_train)) 
         if df_train.loc[i, 'Name'].split(', ', 1)[1].split(' ', 1)[0] == title]))

for title in titles:
    unique_sexes_titles(title)

Title: Master.
{'male'}
Title: Major.
{'male'}
Title: Sir.
{'male'}
Title: Dr.
{'male', 'female'}
Title: Rev.
{'male'}
Title: Mrs.
{'female'}
Title: Col.
{'male'}
Title: Mme.
{'female'}
Title: the
{'female'}
Title: Lady.
{'female'}
Title: Capt.
{'male'}
Title: Ms.
{'female'}
Title: Jonkheer.
{'male'}
Title: Don.
{'male'}
Title: Mr.
{'male'}
Title: Miss.
{'female'}
Title: Mlle.
{'female'}


The assigned sexes seem to match the respective titles. We're going to create a column for the titles.

In [49]:
df_train['Title'] = [name.split(', ', 1)[1].split(' ', 1)[0] for name in df_train['Name']]

We will replace 'the' by 'Countess' for the Countess of Rothes.

In [50]:
df_train.loc[df_train['Title'] == 'the', 'Title'] = 'Countess'

We'll do the same check for the test set.

In [51]:
pd.DataFrame([name.split(', ', 1)[1].split(' ', 1)[0] for name in df_test['Name']]).value_counts()

0      
Mr.        240
Miss.       78
Mrs.        72
Master.     21
Col.         2
Rev.         2
Dr.          1
Dona.        1
Ms.          1
Name: count, dtype: int64

In [52]:
titles = set([name.split(', ', 1)[1].split(' ', 1)[0] for name in df_test['Name']])

def unique_sexes_titles(title):
    print('Title:', title)
    print(set([df_test.loc[i, 'Sex'] for i in range(len(df_test)) 
         if df_test.loc[i, 'Name'].split(', ', 1)[1].split(' ', 1)[0] == title]))

for title in titles:
    unique_sexes_titles(title)

Title: Master.
{'male'}
Title: Dr.
{'male'}
Title: Rev.
{'male'}
Title: Mrs.
{'female'}
Title: Col.
{'male'}
Title: Dona.
{'female'}
Title: Ms.
{'female'}
Title: Mr.
{'male'}
Title: Miss.
{'female'}


This all looks fine as well.

In [53]:
df_test['Title'] = [name.split(', ', 1)[1].split(' ', 1)[0] for name in df_test['Name']]

The first part of the elements of the `Name` column is the surname of the passengers. We will give these surnames a separate column.

In [54]:
df_train['Surname'] = [name.split(', ', 1)[0] for name in df_train['Name']]
df_test['Surname'] = [name.split(', ', 1)[0] for name in df_test['Name']]

We will also create a separate column for the first name.

In [55]:
df_train['Firstname'] = [
    name.split('. ', 1)[1].split(' ')[0].replace('(', '').replace(')', '') for name in df_train['Name']
]
df_test['Firstname'] = [
    name.split('. ', 1)[1].split(' ')[0].replace('(', '').replace(')', '') for name in df_test['Name']
]

One potential issue is that many married women carry their husband's name. The maiden names are recorded between brackets.

In [56]:
names_married = df_test.loc[[i for i in range(len(df_test)) if 'Mrs.' in df_test.loc[i, 'Name']], 'Name']
ind = names_married.index
maiden_names = [name.split('(')[1].split(')')[0].replace('"', '') for name in names_married]
firstnames = [name.split(' ')[0] for name in maiden_names]

It's pretty straightforward extracting the married women's first names. However, some women either didn't give up their maiden names or they kept their maiden names after marriage. This is due to different customs in different cultures when it comes to adopting the husband's name. So we have to ignore those instances when extracting the maiden names for the other women. 

In [57]:
surnames = [maiden_names[i].split(' ')[-1] if '. (' not in list(names_married)[i] 
            else list(names_married)[i].split(',')[0] for i in range(len(names_married))]

Now we need to replace the husbands' names by the maiden names.

In [58]:
df_test.loc[ind, 'Firstname'] = firstnames
df_test.loc[ind, 'Surname'] = surnames

We do the same for the training set.

In [59]:
names_married = df_train.loc[[i for i in range(len(df_train)) if 'Mrs.' in df_train.loc[i, 'Name']], 'Name']
ind = names_married.index
maiden_names = [list(names_married)[i].split('(')[1].split(')')[0].replace('"', '') if '(' in list(names_married)[i]
                else list(names_married)[i].split('Mrs. ')[1].split(' ')[0] for i in range(len(names_married))]
firstnames = [name.split(' ')[0] for name in maiden_names]
surnames = [maiden_names[i].split(' ')[-1] if len(list(names_married)[0].split('Mrs. ')[1].split(' ')) > 1
            else list(names_married)[i].split(',')[0] for i in range(len(names_married))]
df_train.loc[ind, 'Firstname'] = firstnames
df_train.loc[ind, 'Surname'] = surnames

We can now use the names to predict the nationality of the passengers. There is a package named `ethnicolr` that can help to predict the nationality of the individuals. Unfortunately, this package does not let me use it on my computer for some reason. Therefore, we're going to use the results from the `ethnicolr` package shared by another person that worked on this data set. (Source: https://www.kaggle.com/datasets/warrenelder/titanic-passenger-nationalities/data.)

These are the nationalities that are found in the predictions:

In [60]:
set(list(df_train['Nationality']) + list(df_test['Nationality']))

{'African,EastAfrican',
 'African,WestAfrican',
 'CelticEnglish',
 'EastAsian,Chinese',
 'EastAsian,Indochina,Vietnam',
 'EastAsian,Malay,Indonesia',
 'EastAsian,Malay,Malaysia',
 'EastAsian,South Korea',
 'European,French',
 'European,German',
 'European,Italian,Italy',
 'European,Italian,Romania',
 'European,Russian',
 'European,SouthSlavs',
 'Greek',
 'Hispanic,Philippines',
 'Hispanic,Portuguese',
 'Hispanic,Spanish',
 'Muslim,Nubian',
 'Muslim,Pakistanis,Bangladesh',
 'Muslim,Persian',
 'Muslim,Turkic,Turkey',
 'Nordic,Finland',
 'Nordic,Scandinavian,Denmark',
 'Nordic,Scandinavian,Norway',
 'Nordic,Scandinavian,Sweden',
 'SouthAsian'}

## 2.11 Ticket

There is one column that we haven't addressed yet. It's `Ticket` which contains the ticket numbers.

In [61]:
print('#unique ticket numbers in training set:', len(set(df_train['Ticket'])))
print('#unique ticket numbers in test set:', len(set(df_test['Ticket'])))

#unique ticket numbers in training set: 681
#unique ticket numbers in test set: 363


There are 681 unique ticket numbers in the training set (size: 891) and 363 in the test set (size: 418). We've earlier that some people travel together on the same ticket. We also saw that some ticket numbers are completely numeric and some contain letters and other characters.

In [62]:
print('#fully numeric ticket numbers in the training set:', 
      len([i for i in range(len(set(df_train['Ticket']))) if list(set(df_train['Ticket']))[i].isnumeric()]))
print('#fully numeric ticket numbers in the test set:', 
      len([i for i in range(len(set(df_test['Ticket']))) if list(set(df_test['Ticket']))[i].isnumeric()]))

#fully numeric ticket numbers in the training set: 514
#fully numeric ticket numbers in the test set: 265


Most ticket numbers are fully numeric, but a good chunk contains other characters. From what we can see by globally scrolling through the raw data, it seems like the ticket numbers that aren't completely numeric consist of two parts: the prefix and the numeric part. We combine all the prefixes of the training and test set for further investigation.

In [63]:
all_tns = list(set(df_train['Ticket'])) + list(set(df_test['Ticket']))

set([all_tns[i].rsplit(' ', 1)[0] for i in range(len(all_tns)) if not all_tns[i].isnumeric()])

{'A. 2.',
 'A./5.',
 'A.5.',
 'A/4',
 'A/4.',
 'A/5',
 'A/5.',
 'A/S',
 'A4.',
 'AQ/3.',
 'AQ/4',
 'C',
 'C.A.',
 'C.A./SOTON',
 'CA',
 'CA.',
 'F.C.',
 'F.C.C.',
 'Fa',
 'LINE',
 'LP',
 'P/PP',
 'PC',
 'PP',
 'S.C./A.4.',
 'S.C./PARIS',
 'S.O./P.P.',
 'S.O.C.',
 'S.O.P.',
 'S.P.',
 'S.W./PP',
 'SC',
 'SC/A.3',
 'SC/A4',
 'SC/AH',
 'SC/AH Basle',
 'SC/PARIS',
 'SC/Paris',
 'SCO/W',
 'SO/C',
 'SOTON/O.Q.',
 'SOTON/O2',
 'SOTON/OQ',
 'STON/O 2.',
 'STON/O2.',
 'STON/OQ.',
 'SW/PP',
 'W./C.',
 'W.E.P.',
 'W/C',
 'WE/P'}

Some of the unique prefixes actually look like different ways of recording the same thing. For example, it is safe to assume that 'S.C./PARIS', 'SC/PARIS' and 'SC/Paris' are all referring to the same prefix. We could do something about these 'errors', but we won't be using this feature in further analysis, so we will not spend more time on this.

## 3. Summary

We have filled a missing value in the `Fare` column in the test set. In the training set, the port of embarking was missing for two women in the training set that were traveling together. We made an educated guess of where they embarked from looking at passengers with similar ticket and cabin numbers. There were a lot of missing values for the columns `Age` and `Cabin` in both data sets. For age, we marked the passengers for whom the age is not known by assigning 1 in the added column `Missing_Age`, and for the cabin number, we replaced the missing values by 'unknown'. We derived the deck from the cabin numbers and included them in a newly created column, `Deck`. We have removed strange characters from the `Name` column. From this column, we've obtained the honorrific titles of the passengers, separated the first and last names taking into account maiden names of married women and predicted the passengers' ethnic/cultural background. We have not found any other inconsistencies in the data.

In [64]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 17 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         418 non-null    float64
 9   Cabin        418 non-null    object 
 10  Embarked     418 non-null    object 
 11  Nationality  418 non-null    object 
 12  Missing_Age  418 non-null    int64  
 13  Deck         418 non-null    object 
 14  Title        418 non-null    object 
 15  Surname      418 non-null    object 
 16  Firstname    418 non-null    object 
dtypes: float64(2), int64(5), object(10)
memory usage: 55.6+ KB


In [65]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 18 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        891 non-null    object 
 11  Embarked     891 non-null    object 
 12  Nationality  891 non-null    object 
 13  Missing_Age  891 non-null    int64  
 14  Deck         891 non-null    object 
 15  Title        891 non-null    object 
 16  Surname      891 non-null    object 
 17  Firstname    891 non-null    object 
dtypes: float64(2), int64(6), object(10)
memory usage: 

We will save these manipulated datasets so we can use them in further analysis.

In [66]:
df_test.to_csv('preprocessed_testdata.csv', index=False)
df_train.to_csv('preprocessed_traindata.csv', index=False)